In [4]:
import pandas as pd
import numpy as np
import os

In [3]:
!unzip train_label.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_label/Maltese dog/1883.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._1883.jpg  
  inflating: train_label/Maltese dog/1061.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._1061.jpg  
  inflating: train_label/Maltese dog/536.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._536.jpg  
  inflating: train_label/Maltese dog/6257.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._6257.jpg  
  inflating: train_label/Maltese dog/3887.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._3887.jpg  
  inflating: train_label/Maltese dog/2435.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._2435.jpg  
  inflating: train_label/Maltese dog/2637.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._2637.jpg  
  inflating: train_label/Maltese dog/2179.jpg  
  inflating: __MACOSX/train_label/Maltese dog/._2179.jpg  
  inflating: train_label/Maltese dog/5416.jpg  
  inflating: __MACOSX/train_label/Maltese dog/.

In [ ]:
source = 'train_shuffle'
destination = 'train_label'


image2idx = {x[0]: x[1] for x in labels[['image','subclass_index']].itertuples(index=False)}


# gather all files
allfiles = os.listdir(source)
 
# iterate on all files to move them to destination folder
for f in allfiles:
    img_idx = image2idx[f]
    img_label = subclass_mapping[img_idx].split(",")[0]
    
    src_path = os.path.join(source, f)
    dst_path = os.path.join(destination+'/'+img_label, f)
    
    # print(src_path, dst_path)
    
    os.rename(src_path, dst_path)

In [ ]:
for c in subclass_mapping:
    label = subclass_mapping[c].split(",")[0]
    #print( "train_label/" + label);
    os.makedirs( "train_label/" + label)


In [ ]:
subclass_mapping = pd.read_csv("sub_classes_mapping.csv")
superclass_mapping = pd.read_csv("super_classes_mapping.csv")

In [ ]:
subclass_mapping = {x[0]: x[1] for x in subclass_mapping.itertuples(index=False)}
superclass_mapping = {x[0]: x[1] for x in superclass_mapping.itertuples(index=False)}

In [ ]:
labels = pd.read_csv("train_data.csv")
#labels.set_index('image', drop=True, inplace=True)
labels

,image,superclass_index,subclass_index
0,0.jpg,1,63
1,1.jpg,0,24
2,2.jpg,0,74
3,3.jpg,2,72
4,4.jpg,2,43
...,...,...,...
6467,6467.jpg,0,26
6468,6468.jpg,1,22
6469,6469.jpg,2,29
6470,6470.jpg,2,68


In [ ]:
import torch
from vit_pytorch import ViT

# subclass len = 90, super = 3
v = ViT(
    image_size = 16,
    patch_size = 16,
    num_classes = 90,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

img = torch.randn(1, 3, 256, 256)

preds = v(img) # (1, 1000)

90

In [5]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 30.8 MB/s 
     |████████████████████████████████| 5.8 MB 59.0 MB/s 
     |████████████████████████████████| 132 kB 35.4 MB/s 
     |████████████████████████████████| 182 kB 70.7 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
     |████████████████████████████████| 7.6 MB 63.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [68]:
from datasets import load_dataset, Dataset

In [23]:
val = load_dataset("imagefolder", data_dir="train_label", split="train[:10%]")
train = load_dataset("imagefolder", data_dir="train_label", split="train[-90%:]")

Resolving data files:   0%|          | 0/6472 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/6472 [00:00<?, ?it/s]

In [27]:
train

Dataset({
    features: ['image', 'label'],
    num_rows: 5825
})

In [36]:
labels = val.features['label']
labels


89

In [9]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [10]:
def transform(batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = batch['label']
    return inputs

In [81]:
def transform_test(batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in batch['image']], return_tensors='pt')

    return inputs

In [37]:
val_trans = val.with_transform(transform)
train_trans = train.with_transform(transform)

In [44]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [45]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [46]:
from transformers import ViTForImageClassification

labels = train.features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/1ba429d32753f33a0660b80ac6f43a3c80c18938/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "Afghan hound",
    "1": "African chameleon",
    "10": "Chihuahua",
    "11": "Dandie Dinmont",
    "12": "European fire salamander",
    "13": "European gallinule",
    "14": "Gila monster",
    "15": "Japanese spaniel",
    "16": "Komodo dragon",
    "17": "Lakeland terrier",
    "18": "Lhasa",
    "19": "Maltese dog",
    "2": "African crocodile",
    "20": "Pekinese",
    "21": "Rhodesian ridgeback",
    "22": "Scotch terrier",
    "23": "Sealyham terrier",
    "24": "Shih-Tzu",
    "25": "Tibetan terrier",
    "26"

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-jw4169",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_trans,
    eval_dataset=val_trans,
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [49]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5825
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1460
  Number of trainable parameters = 85867097


Step,Training Loss,Validation Loss,Accuracy
100,4.308700,5.638566,0.000000
200,3.999800,5.728307,0.000000
300,3.776800,5.968629,0.000000
400,3.491200,5.897933,0.000000
500,3.358000,6.189960,0.000000
600,3.230200,6.254401,0.000000
700,3.057900,6.196345,0.000000
800,2.927800,6.180874,0.000000
900,2.711500,6.243356,0.000000
1000,2.653800,6.331916,0.001546


***** Running Evaluation *****
  Num examples = 647
  Batch size = 8
Saving model checkpoint to ./vit-base-jw4169/checkpoint-100
Configuration saved in ./vit-base-jw4169/checkpoint-100/config.json
Model weights saved in ./vit-base-jw4169/checkpoint-100/pytorch_model.bin
Image processor saved in ./vit-base-jw4169/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 8
Saving model checkpoint to ./vit-base-jw4169/checkpoint-200
Configuration saved in ./vit-base-jw4169/checkpoint-200/config.json
Model weights saved in ./vit-base-jw4169/checkpoint-200/pytorch_model.bin
Image processor saved in ./vit-base-jw4169/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 8
Saving model checkpoint to ./vit-base-jw4169/checkpoint-300
Configuration saved in ./vit-base-jw4169/checkpoint-300/config.json
Model weights saved in ./vit-base-jw4169/checkpoint-300/pytorch_model.bin
Image processor saved

***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1682873118GF
  train_loss               =       3.0986
  train_runtime            =   0:08:56.93
  train_samples_per_second =       43.395
  train_steps_per_second   =        2.719


In [50]:
metrics = trainer.evaluate(val_trans)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 647
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =        0.0
  eval_loss               =     5.6386
  eval_runtime            = 0:00:05.38
  eval_samples_per_second =    120.155
  eval_steps_per_second   =     15.043


In [51]:
! pip install aimodelshare --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 952 kB 31.6 MB/s 
     |████████████████████████████████| 442 kB 58.6 MB/s 
     |████████████████████████████████| 13.1 MB 59.4 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 24.9 MB 10.5 MB/s 
     |████████████████████████████████| 146 kB 83.0 MB/s 
     |████████████████████████████████| 288 kB 63.2 MB/s 
     |████████████████████████████████| 8.6 MB 55.6 MB/s 
     |████████████████████████████████| 83 kB 2.9 MB/s 
     |████████████████████████████████| 178 kB 75.0 MB/s 
     |████████████████████████████████| 280 kB 73.2 MB/s 
     |████████████████████████████████| 4.5 MB 67.3 MB/s 
     |████████████████████████████████| 132 kB 75.2 MB/s 
     |████████████████████████████████| 308 kB 77.5 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |█████████████████████████████

In [52]:
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/neuralnet_subclasses_competition_data-repository:latest')


Data downloaded successfully.


In [53]:
# Create training data objects 

# Extract images
!unzip "neuralnet_subclasses_competition_data/X_train.zip"
!unzip "neuralnet_subclasses_competition_data/X_test.zip" 

# Create ordered list of filepaths 
train_filepaths = [('/content/train_shuffle/' + str(i) + '.jpg') for i in range(0, 6472)]
test_filepaths = [('/content/test_shuffle/' + str(i) + '.jpg') for i in range(0, 9127)]

# Read in y_train data 
import pandas as pd 
y_train = pd.read_csv("neuralnet_subclasses_competition_data/y_train.csv")

Streaming output truncated to the last 5000 lines.
  inflating: test_shuffle/8701.jpg   
  inflating: test_shuffle/1057.jpg   
  inflating: test_shuffle/6738.jpg   
  inflating: test_shuffle/7426.jpg   
  inflating: test_shuffle/8715.jpg   
  inflating: test_shuffle/3640.jpg   
  inflating: test_shuffle/3898.jpg   
  inflating: test_shuffle/5231.jpg   
  inflating: test_shuffle/2238.jpg   
  inflating: test_shuffle/4891.jpg   
  inflating: test_shuffle/5557.jpg   
  inflating: test_shuffle/4649.jpg   
  inflating: test_shuffle/3126.jpg   
  inflating: test_shuffle/7340.jpg   
  inflating: test_shuffle/8073.jpg   
  inflating: test_shuffle/1731.jpg   
  inflating: test_shuffle/2210.jpg   
  inflating: test_shuffle/4661.jpg   
  inflating: test_shuffle/7368.jpg   
  inflating: test_shuffle/6076.jpg   
  inflating: test_shuffle/1719.jpg   
  inflating: test_shuffle/6710.jpg   
  inflating: test_shuffle/3668.jpg   
  inflating: test_shuffle/4107.jpg   
  inflating: test_shuffle/5219.jpg   

In [78]:
test_data = load_dataset("imagefolder", data_dir='test_shuffle')


Resolving data files:   0%|          | 0/9127 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [92]:
test_data

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 9127
    })
})

In [ ]:
test_trans = test_data.with_transform(transform_test)

preds = trainer.predict(test_data['train'])